In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path = "/content/drive/MyDrive/hindi_dataset/"
files = os.listdir(path)[:9]
print(files)

classes = {'cha':0,'ka':1,'ma':2,'na':3,'pa':4,'pha':5,'ra':6,'ta':7,'thaa':8,'wa':'9'}

['ka', 'pha', 'ma', 'pa', 'ta', 'na', 'wa', 'thaa', 'ra']


In [ ]:
import cv2

X = []
Y = []

for cl in classes:
    pth = path+cl
    for img_name in os.listdir(pth):
        img = cv2.imread(pth+"/"+img_name,0)
        X.append(img)
        Y.append(classes[cl])
print("Dataset created successfully")

Dataset created successfully


In [ ]:
pd.Series(Y).value_counts()

9    2010
8    2000
7     250
dtype: int64

In [ ]:
X[0].shape

(32, 32)

In [ ]:
print(type(X))
X = np.array(X)
Y = np.array(Y)
print(type(X))

<class 'list'>
<class 'numpy.ndarray'>


In [ ]:
fig = px.imshow(X[2001], color_continuous_scale="gray", title=f"Sample image (Class: {Y[2001]})")
fig.show()


In [ ]:
X_new = X.reshape(len(X),-1)
print(X_new.shape)
print(Y.shape)

(4260, 1024)
(4260,)


In [ ]:
print(X.shape)
print(X.ndim)
print(X_new.ndim)

(4260, 32, 32)
3
2


In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X_new,Y,
                                               test_size=.20, random_state=10)

In [ ]:
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

(3408, 1024) (3408,)
(852, 1024) (852,)


In [ ]:
print(xtrain.max())
print(xtest.max())

255
255


In [ ]:
x_train = xtrain/255
x_test = xtest/255
print(x_train.max())
print(x_test.max())

1.0
1.0


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
print(x_train.shape, x_test.shape)
pca = PCA(.96)
xtrain = pca.fit_transform(x_train)
xtest = pca.transform(x_test)
print(xtrain.shape, xtest.shape)
print(pca.n_components)
print(pca.n_features_)

(3408, 1024) (852, 1024)
(3408, 168) (852, 168)
0.96
1024


/usr/local/lib/python3.9/dist-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute `n_features_` was deprecated in version 1.2 and will be removed in 1.4. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
pca_df = pd.DataFrame(xtrain, columns=[f"PC{i+1}" for i in range(xtrain.shape[1])])
pca_df["Class"] = ytrain

fig = px.scatter(pca_df, x="PC1", y="PC2", color="Class", title="Scatter plot of PCA components")
fig.show()


In [ ]:
ytest[:10]

array(['9', '8', '7', '8', '8', '9', '9', '8', '8', '8'], dtype='<U21')

In [ ]:
lm = LogisticRegression()
lm.fit(xtrain, ytrain)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
tr_pred = lm.predict(xtrain)
ts_pred = lm.predict(xtest)

In [ ]:
print("Training Score", accuracy_score(ytrain, tr_pred)*100)
print("Testing Score", accuracy_score(ytest, ts_pred)*100)

Training Score 97.21244131455398
Testing Score 94.71830985915493


In [ ]:
fig = go.Figure(data=[go.Bar(x=['Training Accuracy', 'Testing Accuracy'],
                             y=[accuracy_score(ytrain, tr_pred)*100, accuracy_score(ytest, ts_pred)*100],
                             text=[f'{accuracy_score(ytrain, tr_pred)*100:.2f}%', f'{accuracy_score(ytest, ts_pred)*100:.2f}%'],
                             textposition='auto',
                             textfont=dict(color='white', size=14),
                             marker=dict(color=['#636EFA', '#EF553B']))])
fig.update_layout(title='Accuracy of Logistic Regression Model',
                  xaxis_title='Type of Accuracy',
                  yaxis_title='Accuracy (%)',
                  font=dict(size=18),
                  showlegend=False)
fig.show()
